In [79]:
import pandas as pd

In [80]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['46A','47A','78','LCBK']]
display(df)

,46A,47A,78,LCBK
3,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA..."
4,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)"
5,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...
7,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
9,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"THE MIZUHO BANK,LTD.,HEAD OFFICE"
...,...,...,...,...
8403,1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS IND...,"1. ALL DOCUMENTS MUST BE DATED, SIGNED AND ISS...",+THE AMOUNT OF EACH DRAWING UNDER THIS CREDIT ...,AGRICULTURAL BANK OF \nCHINA LTD
8413,+ COMMERCIAL INVOICE IN 3 ORIGINAL\r\n+ 2/3 SE...,+ T.T.REIMBURSEMENT : PROHIBITED\r\n+ INSURANC...,"ALL DOCS TO BE SENT DIRECTLY TO US (3-10-19, M...",SUMITOMO MITSUI BANKING \nCORPORATION JAPAN
8414,+SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 OR...,+KINDLY ASSIST US TO RELAY THIS CREDIT TO CITI...,+TO NEGOTIATING BANK ONLY : PLEASE FORWARD ALL...,"CTBC BANK CO.,LTD."
8415,+SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 OR...,+KINDLY ASSIST US TO RELAY THIS CREDIT TO CITI...,+TO NEGOTIATING BANK ONLY : PLEASE FORWARD ALL...,"CTBC BANK CO.,LTD."


In [81]:
for i in ['46A','47A','78','LCBK']:
    print(df[i].str.contains('x000D').value_counts())

False    3641
Name: 46A, dtype: int64
False    3590
Name: 47A, dtype: int64
False    3554
Name: 78, dtype: int64
False    3643
Name: LCBK, dtype: int64


In [82]:
df['46A'].values[0]

"1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND 3 COPIES.2. FULL SET OF SIGNED 'CLEAN ON BOARD' OCEAN BILLS OF LADINGMADE OUT TO ORDER OF LETTER OF CREDIT ISSUING BANK MARKEDFREIGHT PREPAID EVIDENCING SHIPMENT OF MERCHANDISE DESCRIBEDABOVE. BILLS OF LADING MUST STATE FULL NAME AND ADDRESS OF BOTHAPPLICANT AND ICICI BANK LTD, SHALIMAR TOWER, 31/54 M.G.MARG,HAZRATGANJ, LUCKNOW 226001,UTTAR PRADESH, INDIA AS PARTIES TOBE NOTIFIED. SHORT FORM,FREIGHT FORWARDERS AND LASHBILLS OF LADING ARE NOT ACCEPTABLE.14 DAYS FREE DETENTION TIMESHOULD BE GIVEN AT DISCHARGE PORT AND THE SAME SHOULD BEMENTIONED EITHER ON THE BILL OF LADING OR BY A SEPARATECERTIFICATE FROM SHIPPING COMPANY OR FROM THEIR AGENTS MUSTACCOMPANY ORIGINAL DOCUMENTS.3.MARINE INSURANCE POLICY OR CERTIFICATE (1 ORIGINAL PLUS 1COPY) DATED NOT LATER THAN THE DATE OF BILLS OF LADING SIGNEDAND ISSUED BY INSURANCE COMPANY MADE TO ORDER AND BLANKENDORSED FOR 110 PCT OF CIF VALUE OF GOODS SUPPLIED, COVERINGINSTITUTE CARGO CLAUSE (A), INST

In [83]:
import numpy as np

import re

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    return x

def get_bank(text):
    text = str(text)
    text = preprocess(text)
    keywords = ['TO ORDER OF','TO THEORDER OF','TO THE ORDER OF','TOTHE ORDER OF','TO THE ORDER+OF','TOORDER OF']
    for i in keywords:
        if i in text:
            idx = text.split(i)[1].find('BANK')
            result = preprocess(text.split(i)[1][:idx+len('BANK')])
            if 'BANK' in result:
                return result
            else:
                return np.nan 
        else:
            return np.nan

In [84]:
df['46A'] = df['46A'].apply(preprocess)
df['47A'] = df['47A'].apply(preprocess)
df['78'] = df['78'].apply(preprocess)
df['LCBK'] = df['LCBK'].apply(preprocess)

In [85]:
df

,46A,47A,78,LCBK
3,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,1 ALL DOCUMENTS MUST BE IN ENGLISH2 ALL DOCUME...,UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT ...,ICICI BANK LTDSHALIMAR TOWER 3154 MGMARGHAZRAT...
4,1 SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4 ...,A ALL DOCUMENTS AND DRAFTS IF CALLED FOR UNDER...,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,NATIONAL COMMERCIAL BANK THEHEAD OFFICE
5,1SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AND...,ALL DOCUMENTS MUST MADE IN ENGLISHALL DOCUMENT...,UPON RECEIPT OF ALL DOCUMENTS SENT TO USVIETNA...,VIET NAM BANK FOR AGRICULTURE AND RURAL DEVELO...
7,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,ALL DRAFTS DRAWN HEREUNDER MUST BE MARKED DRAW...,ALL DOCUMENTS INCLUDING BENEFICIARYS DRAFTS MU...,THE MIZUHO BANKLTDHEAD OFFICE
9,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,ALL DRAFTS DRAWN HEREUNDER MUST BE MARKED DRAW...,ALL DOCUMENTS INCLUDING BENEFICIARYS DRAFTS MU...,THE MIZUHO BANKLTDHEAD OFFICE
...,...,...,...,...
8403,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDI...,1 ALL DOCUMENTS MUST BE DATED SIGNED AND ISSUE...,THE AMOUNT OF EACH DRAWING UNDER THIS CREDIT M...,AGRICULTURAL BANK OF CHINA LTD
8413,COMMERCIAL INVOICE IN 3 ORIGINAL 23 SET OF CLE...,TTREIMBURSEMENT PROHIBITED INSURANCE IS TO BE...,ALL DOCS TO BE SENT DIRECTLY TO US 3-10-19 MIN...,SUMITOMO MITSUI BANKING CORPORATION JAPAN
8414,SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 ORI...,KINDLY ASSIST US TO RELAY THIS CREDIT TO CITIT...,TO NEGOTIATING BANK ONLY PLEASE FORWARD ALL D...,CTBC BANK COLTD
8415,SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 ORI...,KINDLY ASSIST US TO RELAY THIS CREDIT TO CITIT...,TO NEGOTIATING BANK ONLY PLEASE FORWARD ALL D...,CTBC BANK COLTD


In [86]:
df['46A'].values[0]

'1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND 3 COPIES2 FULL SET OF SIGNED CLEAN ON BOARD OCEAN BILLS OF LADINGMADE OUT TO ORDER OF LETTER OF CREDIT ISSUING BANK MARKEDFREIGHT PREPAID EVIDENCING SHIPMENT OF MERCHANDISE DESCRIBEDABOVE BILLS OF LADING MUST STATE FULL NAME AND ADDRESS OF BOTHAPPLICANT AND ICICI BANK LTD SHALIMAR TOWER 3154 MGMARGHAZRATGANJ LUCKNOW 226001UTTAR PRADESH INDIA AS PARTIES TOBE NOTIFIED SHORT FORMFREIGHT FORWARDERS AND LASHBILLS OF LADING ARE NOT ACCEPTABLE14 DAYS FREE DETENTION TIMESHOULD BE GIVEN AT DISCHARGE PORT AND THE SAME SHOULD BEMENTIONED EITHER ON THE BILL OF LADING OR BY A SEPARATECERTIFICATE FROM SHIPPING COMPANY OR FROM THEIR AGENTS MUSTACCOMPANY ORIGINAL DOCUMENTS3MARINE INSURANCE POLICY OR CERTIFICATE 1 ORIGINAL PLUS 1COPY DATED NOT LATER THAN THE DATE OF BILLS OF LADING SIGNEDAND ISSUED BY INSURANCE COMPANY MADE TO ORDER AND BLANKENDORSED FOR 110 PCT OF CIF VALUE OF GOODS SUPPLIED COVERINGINSTITUTE CARGO CLAUSE A INSTITUTE WAR CLAUSE CARGO AND

In [87]:
df['47A'].values[0]

'1 ALL DOCUMENTS MUST BE IN ENGLISH2 ALL DOCUMENTS MUST MENTION OUR LC NUMBER AND DATE3 DISCREPANCY FEE OF USD 100- PER SET OF DISCREPANT DOCUMENTSPRESENTED WILL BE DEDUCTED FROM BILL PROCEEDS 4 NEGOTIATING BANK MUST SEND ORIGINAL SET OF DOCUMENTS IN ONELOT TO THE FOLLOWING ADDRESS BY COURIERICICI BANK LTD SHALIMAR TOWER 3154 MGMARG HAZRATGANJLUCKNOW 226001UTTAR PRADESH INDIA5 NOTWITHSTANDING THE CONTENT OF ARTICLE 16 OF UCP 600  INTHE EVENT THAT WE OBSERVE DISCREPANCIES IN DOCUMENTS WE SHALLPROVIDE OUR NOTICE OF REFUSAL IN ACCORDANCE THEREWITH AND IF WEELECT TO CONTACT THE APPLICANT FOR A WAIVER AND WE RECEIVE ANACCEPTABLE WAIVER WE SHALL EFFECT SETTLEMENT ACCORDING TO THELC TERMS AND THE DOCUMENTS WILL BE RELEASED WITHOUT FURTHERNOTICE TO YOU UNLESS WE ARE IN RECEIPT OF YOUR INSTRUCTION TOTHE CONTRARY PRIOR TO SUCH RELEASE6 THIS LC WOULD BE SUBJECT TO US EU UN INDIA ANDOR OTHERSANCTIONS INCLUDING DUE TO USE OF A CORRESPONDENT BANKCONSEQUENTLY PROCESSING OF THIS LC WOULD BE SUBJECT TO

In [88]:
df['78'].values[0]

'UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT OUR COUNTERSWE WILL AUTHORISE THE NEGOTIATING BANK TO CLAIMREIMBURSEMENT FROM THE REIMBURSING BANK REIMBURSEMENTUNDER THIS LC IS SUBJECT TO URR 725 OF ICC EXCEPT AS OTHERWISE EXPRESSLY STATED THIS CREDIT ISSUBJECT TO THE UNIFORM CUSTOMS AND PRACTICE FOR DOCUMENTARYCREDITS 2007 REVISION ICC PUB NO600 ALL OTHER COMMUNICATIONS UNDER THIS LC MAY BEDIRECTLY SENT TO US ON SWIFT ICICINBBCTS'

In [89]:
df[['LCBK']]

,LCBK
3,ICICI BANK LTDSHALIMAR TOWER 3154 MGMARGHAZRAT...
4,NATIONAL COMMERCIAL BANK THEHEAD OFFICE
5,VIET NAM BANK FOR AGRICULTURE AND RURAL DEVELO...
7,THE MIZUHO BANKLTDHEAD OFFICE
9,THE MIZUHO BANKLTDHEAD OFFICE
...,...
8403,AGRICULTURAL BANK OF CHINA LTD
8413,SUMITOMO MITSUI BANKING CORPORATION JAPAN
8414,CTBC BANK COLTD
8415,CTBC BANK COLTD


In [90]:
table = pd.DataFrame()
table['string_X_train'] = df['46A']+' '+df['47A']+' '+df['78']
table['Y_label'] = df['LCBK']
table

,string_X_train,Y_label
3,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,ICICI BANK LTDSHALIMAR TOWER 3154 MGMARGHAZRAT...
4,1 SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4 ...,NATIONAL COMMERCIAL BANK THEHEAD OFFICE
5,1SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AND...,VIET NAM BANK FOR AGRICULTURE AND RURAL DEVELO...
7,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,THE MIZUHO BANKLTDHEAD OFFICE
9,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,THE MIZUHO BANKLTDHEAD OFFICE
...,...,...
8403,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDI...,AGRICULTURAL BANK OF CHINA LTD
8413,COMMERCIAL INVOICE IN 3 ORIGINAL 23 SET OF CLE...,SUMITOMO MITSUI BANKING CORPORATION JAPAN
8414,SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 ORI...,CTBC BANK COLTD
8415,SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 ORI...,CTBC BANK COLTD


In [91]:
df = table
df['string_X_train'].values[0]

'1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND 3 COPIES2 FULL SET OF SIGNED CLEAN ON BOARD OCEAN BILLS OF LADINGMADE OUT TO ORDER OF LETTER OF CREDIT ISSUING BANK MARKEDFREIGHT PREPAID EVIDENCING SHIPMENT OF MERCHANDISE DESCRIBEDABOVE BILLS OF LADING MUST STATE FULL NAME AND ADDRESS OF BOTHAPPLICANT AND ICICI BANK LTD SHALIMAR TOWER 3154 MGMARGHAZRATGANJ LUCKNOW 226001UTTAR PRADESH INDIA AS PARTIES TOBE NOTIFIED SHORT FORMFREIGHT FORWARDERS AND LASHBILLS OF LADING ARE NOT ACCEPTABLE14 DAYS FREE DETENTION TIMESHOULD BE GIVEN AT DISCHARGE PORT AND THE SAME SHOULD BEMENTIONED EITHER ON THE BILL OF LADING OR BY A SEPARATECERTIFICATE FROM SHIPPING COMPANY OR FROM THEIR AGENTS MUSTACCOMPANY ORIGINAL DOCUMENTS3MARINE INSURANCE POLICY OR CERTIFICATE 1 ORIGINAL PLUS 1COPY DATED NOT LATER THAN THE DATE OF BILLS OF LADING SIGNEDAND ISSUED BY INSURANCE COMPANY MADE TO ORDER AND BLANKENDORSED FOR 110 PCT OF CIF VALUE OF GOODS SUPPLIED COVERINGINSTITUTE CARGO CLAUSE A INSTITUTE WAR CLAUSE CARGO AND

In [92]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X_train'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2
3,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,ICICI BANK LTDSHALIMAR TOWER 3154 MGMARGHAZRAT...,-1,83
4,1 SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4 ...,NATIONAL COMMERCIAL BANK THEHEAD OFFICE,-1,38
5,1SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AND...,VIET NAM BANK FOR AGRICULTURE AND RURAL DEVELO...,-1,65
7,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,THE MIZUHO BANKLTDHEAD OFFICE,-1,28
9,SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDICA...,THE MIZUHO BANKLTDHEAD OFFICE,-1,28


In [93]:
df = df[df['string_Y_1']!=-1]
df.shape

(1592, 4)

In [94]:
df

,string_X_train,Y_label,string_Y_1,string_Y_2
16,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,1860,1876
22,SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICAT...,CTBC BANK CO LTD,4924,4940
23,SIGNED COMMERCIAL INVOICE IN 2 ORIGINALINDICAT...,CTBC BANK CO LTD,4924,4940
30,1SIGNED COMMERCIAL INVOICE IN DUPLICATE SHOWIN...,CHANG HWA COMMERCIAL BANK LTD,5345,5374
41,1FULL SET OF ORIGINALS AND 2 NON-NEGOTIABLE CO...,STANDARD CHARTERED BANK,1596,1619
...,...,...,...,...
8264,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,2134,2150
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,2134,2150
8274,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,2132,2148
8275,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,2132,2148


In [95]:
for i in range(10):
    print(df['string_X_train'].values[i][df['string_Y_1'].values[i]:df['string_Y_2'].values[i]],"|",df['Y_label'].values[i])

CTBC BANK CO LTD | CTBC BANK CO LTD
CTBC BANK CO LTD | CTBC BANK CO LTD
CTBC BANK CO LTD | CTBC BANK CO LTD
CHANG HWA COMMERCIAL BANK LTD | CHANG HWA COMMERCIAL BANK LTD
STANDARD CHARTERED BANK | STANDARD CHARTERED BANK
STANDARD CHARTERED BANK | STANDARD CHARTERED BANK
STANDARD CHARTERED BANK | STANDARD CHARTERED BANK
AXIS BANK LIMITED | AXIS BANK LIMITED
AXIS BANK LIMITED | AXIS BANK LIMITED
AXIS BANK LIMITED | AXIS BANK LIMITED


In [96]:
df.to_csv('../data/preprocess_for_SQUAD_銀行.csv')

In [97]:
find = df['string_X_train'].apply(get_bank).dropna(axis=0)
find

56          AXIS BANK
57          AXIS BANK
58          AXIS BANK
128     INDUSIND BANK
175         AXIS BANK
            ...      
8264     ISSUING BANK
8266     ISSUING BANK
8274     ISSUING BANK
8275     ISSUING BANK
8299     ISSUING BANK
Name: string_X_train, Length: 295, dtype: object

In [98]:
len(find)/len(df)

0.18530150753768845